Oct 18 - Work in progress <br>
References:<br>
https://int8.io/monte-carlo-tree-search-beginners-guide/<br>
https://www.geeksforgeeks.org/ml-monte-carlo-tree-search-mcts/<br>
https://www.reddit.com/r/singularity/comments/1cmcmy5/monte_monte_carlo_tree_search_with_llms_is_the/ <br>
Search: Games, Minimax, and Alpha-Beta: https://ocw.mit.edu/courses/6-034-artificial-intelligence-fall-2010/resources/lecture-6-search-games-minimax-and-alpha-beta/
https://courses.cs.washington.edu/courses/cse599i/18wi/resources/lecture19/lecture19.pdf<br>
applications illustrated: https://www.informs-sim.org/wsc18papers/includes/files/021.pdf<br>
tic-tak-toe: https://stackoverflow.com/questions/49456415/monte-carlo-tree-search-tic-tac-toe-poor-agent <br>

some leetcode adaptations: https://www.perplexity.ai/search/any-problems-at-leetcode-that-OgI491GFTbKDPRnDknJCUg


#MCTS
## What is it for?
- To create a tree structure search algorithm that balances depth and breadth (exploitation and exploration)


## How it works?
- When traveling to each node & staying at certain board state, you simulate the expected returns if traveling a given path to the end, and return rewards to each node at that path.
- You should probably estimate current resources, too.
- The resources and rewards may or may not be the same thing. Likely you may gain resources as rewards, but very likely they are different things.
- Return: MCTS is like an assistant. It takes in current board state and returns you a recommendation for the next move, according to many times of simulations (Not as you thought that it would give you a whole path).

### Selection:

### Expansion:
The action of exploring untried moves and adding them to your list of children. Untried moves are those empty positions on the board.

*For MCTS, you always expand first untile there is no node to be added to your children and then you select best one as the next move. So expand --> select.

*expansion VS selection: two different moves, connecting new nodes VS selecting your best child;
*exploration VS exploitation: two different strategies, exploring new nodes VS go down the current path.

### Simulation:
For a chosen node, you traverse all the way till the end to reap all the rewards along the way.

What is one round of simulation? Does it do the same as one run of MCTS?
No.
- One round of simulation starts with a actual board state to a simulated board state where you receive an end state (no further move can be made)
- In such a simulation, each node in the current copy of board, will connect parent and children, will go down a particular path (simulation step)
- For each iteration of MCTS (a copy board state):
  - selection: you select the best child node for current parent, if the current node is fully expanded and it's not terminal, yet.
  - expansion you expand a new node as long as there is still child to be added.
  - simulation: you start with the newly expanded node and simulate current path to the end as long as (while) there is untried nove left and no winner declared
  - backpropagation: you return the stats of this simulation to the nodes traversed in this simulation.

Key things to be differentiated between selection, expansion, and simulation:
- Selection is to exploit and expansion to explore.
- Expansion connects one new node a time, and simulation continues that new node to the end of this path.
- Selection also down the path of selected node, but with each move calculated whereas simulation does things randomly. You'd hope this randomization to give you a bit signal (by BP) that you can used for future moves.
- Simulation is about exploitation whereas selection is about a balance between exploration and exploitation.

### Backpropagation:
return the stats (steps/resources + rewards) to parent nodes such that at parent nodes they may decide an optimal next move (for selection). It was part of, however the opposite route of the simulation. Simulation goes down the rabbithole, and GP goes out of it and back.


### Overall MCTS workflow
- I had some fundamental misunderstanding of its workflow originally:
  - I thougt it was a class. It probably does not work that way because a class is for algorithms + data structure where you store something. But a MCTS is just an algorithm, a stragegy for your game. It does not hold data like you would need for a board or a node.
  - I thought you'd define 4 functions for 4 steps. It was not wrong, but just not necessary. It's perfectly fine without them.
  - I could not differentiate MCTS workflow from what's in node/board. What's about the rules and data structure of the game should be in either node or board, whereas MCTS is about the strategy.
  - I did not realize with a MCTS you input current board state and outut a recommended move. In retrospect, this is actually most intuitive.
  - I did not know that you create copies of board states on which you would run 4 steps (not only simulation!).
  - I did not know that any changes on references (associated to each iteration/copy of board state) to the root nodes (what you are assigned when starting the MCTS) will also apply to the root nodes. Like the stats (wins/visits), and children nodes connected to.


- Tips for connecting diff class/functions together
  -

## Board and Node
- Node: Each node is a TreeNode, where you have attributes and methods can be applied to each node. Please be reminded that everything you'd apply to a node, like a list, is independent from a different node.
- Board: where you set up result, board state,
- The tricky part for this game is though, you have to get a copy of _current board state_ for each simulation, the real deep copy that does not affect one another.
- Connections between Node and Board: When you make a move, your node changes (you select next move based on UCB1) and your board changes, too (the current board state).
- Mind that SELECT and EXPAND are very different operations. SELECT makes next move, where EXPAND adds a potential child to current parent thereafter you may do simulation with.
- UCB1: an algorithm, or actually just one sorting funciton that to balence exploration and exploitation. Two terms: exploitation term that to calculate wins/visits of current path; and exploration term that to encourage exploring different paths where you've visited less and are less certain about their wins/visits.

What is really a board? Is that a static board state, as you've regarded as, or a dynamic one?
If a dynamic one, where to draw the boundary (what is a board, in contrast to other boards?)
- I think a board should represent a static board state, with moves actually made, not simulated.
- In contrast, a copy represents a board state that starts with the actual one, but has speculated moves on there.
- so when you define a board state, you have # of nodes defined, each represent a actual moved position on there.

What is really a node if it does not represent a move (bc MCTS returns a move)
- I think a node represent a position, travelled or untravelled (but extended, and then simulated), belongs to a give board state. <br>
- The above understanding is not complete. I think you had this conception that a node is a defined position on the board state. This is only true when you define a node with `node = Node(idx = idx)`. But a node can work perfect fine if `idx=None`. So a more nuanced understanding of a node is it's a position on current board state but it's idx can be unknown. A node would be just a node structure that you can have all its nice attributes and methods. In this TTT example, you may connect a node to its parent & children (both speak for its relative positions), you give it an idx, a board state with `(0,1;x,o)` all indictaed. However, any of them can be set as `None`.
  - So yes it should be good to have idx of a node, but it's fine if not. And its idx can be either user-identified, or randomly choosen in `EXPAND`. <br>

What is a fully expanded node?
- a fully expanded node means all untravelled nodes are added to it as children nodes and there is no need to further expand. We can now explore/select best node out of all children

Is a node configurated differently in different copies and different board states?
- visit & win counts are accumulated through many rounds if simulations.
- for this reason (visit & win counts), I think node is defined independent from board states/copy states. There should be 9 nodes defined, period.
- However, in different simulations (and by extension, different copies), node may be modified by which parent/children they are connected, too.
- In a pre-defined/user-defined board state, node positions are defined, but node connections are not. They are just on board states and you do not know how do they reach that state.
- When you start a new copy of current board state, self.children, a.k.a node connection configuration, should be empty.

## TreeNode data structure
- just like link

## Game/RL
- Design of reward function: yes it should be updated back to every single node, but the calculation is at the final result, which is why it's called "backpropogation". I am writting this because I misunderstood reward function in TTT game is reward = 1 2 3 when you get 1 2 3 in a line. It does not work that way. It should be calculated when you get a 3 in simulation, and then you make the way back to the nodes.
- WHen do you need a reward function?
  - When the steps are more than 2 (otherwise you do move?), and the space for exploration is large and you will need to make simulation and allocate your rewards to those winning strategies in the simulation.
- Policy: I think it's wrong to have a policy up here. Beacuse Node is to deal with attributes and actions on node. I know you wanted it to move "according to certain policy", but it actually just basic "move". Your intended policy should go with MCTS.



## Why it may work better or worse than other methods in diff areas?
- MCTS should work better in scenarios where you cannot traverse all pathes (potentially your space of moves is enormously large and even infinite). Here MCTS's random search may guide the decisions between exploration and exploitation.

## Where things may get tricky?
- It seems MCTS is computationally demanding
-

Why does it even work in many applications?
-
- Computation would not be a limitation for employijng such strategy in reality. In other words, simulation and computation enable calculaton of best strategy. <br><br><br>


The main concept of monte carlo tree search is a search. Search is a set of traversals down the game tree. Single traversal is a path from a root node (current game state) to a node that is not fully expanded. Node being not-fully expanded means at least one of its children is unvisited, not explored. Once not fully expanded node is encountered, one of its unvisited children is chosen to become a root node for a single playout/simulation. The result of the simulation is then propagated back up to the current tree root updating game tree nodes statistics. Once the search (constrained by time or computational power) terminates, the move is chosen based on the gathered statistics.

Lots of dots not clearly connected together, right?

Let’s try to ask crucial questions regarding the simplified description above to slowly understand all the pieces:

what are expanded or not fully unexpanded game tree nodes?
what it means ‘to traverse down‘ during search? How is the next (child) node selected?
what is a simulation?
what is the backpropagation?
what statistics are back-propagated and updated in expanded game tree nodes ?
How is the final move even chosen?

In [ ]:
# Pseudocode for MCTS (not using a class bc self. not clean )
resources = None
rewards = dict(nodes:rewards) # dict where a reward is associated to a specific node

# overall function
def forward(resources, node):
  while resources:


# Step 1 Selection: the process of deciding next move(node) based on expected rewards
def selection(nodes, rewards):   # I expect nodes to be a tree structure but rewards to be a dict: node:reward
  best_child = best(nodes)  # basically a for loop to go through all nodes and rewards I think
  return best_child

# Step 2 Expansion: the action of creating a node/making a move
def expansion(nodes):
  if resources == 0:
    return None
  cur.next = best_child
  cur = cur.next
  return cur # # NEP I could not really differentiate this from the selection, except moving the node.

# Step 3 Simulation: to calculating expected rewards if you were to exploit current node/path
def simulation(node, rewards):  # Assuming that this is to cal expected return of the current path if you exploit it.
  simulated_rewards = num_nodes_in_simulation * averate_rewards_of_the_path
  return simulated_rewards

# Step 4 Backpropagation: return the stats to the parent node (how many nodes and how much expected rewards)
def backpropagation():
  return stats # signals back to the parents  dict(node:reward)

# Helper functions

# Strategies for expansion
def strategies(nodes):
  # default   # we start with a random strategy
  pick_randome(node.children)

  # best strategy (???)
  reward_new_path = simulation(path, reward)
  reward_best_path = list(self.best_path.values).sum()
  if ward_new_path > reward_best_path:
    self.best_path = new_path # where you got this new path?
  return best_path

# Monitoring steps and rewards

# Monitoring resources
def resource_monitor():   # it's called every time the agent makes a move.
  if resources ==0:
    return "Game Over"
  resource -= 1   # there is no return so far, but when you call this global variable it always tells you current remain resources.

# Monitoring rewards
def rewards monitor():

  # current rewards
  dict(node:rewards)

  # expecter rewards based on simulations
  dict(node: expected_rewards)


SyntaxError: expected ':' (<ipython-input-1-6566e6b25141>, line 12)

In [ ]:
# A pseudocode from https://int8.io/monte-carlo-tree-search-beginners-guide/, with annotation
# Overall function: selection - expansion - simulation - backpropagation - selection, until a optimal path is found.
def monte_carlo_tree_search(root):
    while resources_left(time, computational_power):  # Continue the process until resources (time or computational power) are exhausted
        leaf = traverse(root)  # Step 1: Selection - traverse the tree to select a leaf node (unexplored node)
        simulation_result = rollout(leaf)  # Step 2: Simulation - simulate a random play from the leaf node to a terminal state
        backpropagate(leaf, simulation_result)  # Step 3: Backpropagation - update the stats of all the nodes from the leaf back to the root
    return best_child(root)  # Step 4: Return the best child (the most promising move) of the root based on the number of visits or score
    # NEP is best_child(root) a path or a node
# Step 1: Selection - traverse the tree to select a leaf node (unexplored node)
def traverse(node):
    while fully_expanded(node):  # Keep traversing the tree as long as the node is fully expanded (all children have been visited)
        node = best_ucb(node)  # Choose the child with the best UCT (Upper Confidence Bound applied to Trees) score for exploration
    return pick_unvisited(node.children) or node  # If there are unvisited children, return one. Otherwise, return the current node (could be terminal)
    #NEP what is UCT and do we need to write a UCT function in real code?  # NEP Do we have a function to monitor unvisited node?

# Step 2: Expansion (make a move in real games)
def expand(node):
    if not fully_expanded(node):
        return pick_unvisited(node.children)  # Expand by adding a new unvisited node
    return node  # If already fully expanded, return the current node

# Step 3: Simulation - simulate a random game (play) from the node until a terminal state is reached
def rollout(node):
    while non_terminal(node):  # Keep going until you reach a terminal node (end of the game)
        node = rollout_policy(node)  # Apply a rollout policy (typically random) to pick the next move/child
    return result(node)  # Return the result of the game (win, loss, or draw)

# strategy used in simulation
def rollout_policy(node):  # A simple strategy for random simulation, picks a random child (move) from the current node
    return pick_random(node.children)

# Step 3: Backpropagation - propagate the result of the simulation back up the tree
def backpropagate(node, result):
   if is_root(node):  # Stop when we reach the root of the tree
       return
   node.stats = update_stats(node, result)  # Update the node's stats (e.g., number of visits, win/loss score) based on the result
   backpropagate(node.parent)  # Recursively backpropagate the result to the parent node
   # NEP are results & stats the same thing?
def best_child(node):  # Step 4: Selection - choose the child with the best stats (e.g., highest visit count or average win rate)
    return pick_child_with_highest_visits(node.children)  # Usually, this is based on the number of visits or average reward (score)
    # NEP I didn't cound whether you would decide best child according to visit count.

# Monte Carlo Tree Search Tic-Tac-Toe

Housekeeping


Steps
1/ A board class to reflect the current state of the board, attributes and potential actions you can take on the boards.
  - input: actions on the nodes (changes on board state thereafter)
  - attributes: np array for current board states [2,3,3]
  - actions: change states, return results,
  - output: board states

2/ A node class to track the agents: what set of actions each has, how much rewards and resources they would have after taking certain actions.
  - inputs:
  - actions: move,
  - attributes: board states, pathes, rewards, resources,
  - output: final pathes

3/ MCTS function/class that implement the 4 steps of MCTS to execute TTT. Unclear about function/class for now. The reson that this is seperate from board/node is surely you can have different strategies for the same game. MCTS is only one of them.


Notice:



In [130]:
import numpy as np
import random

In [131]:
# NEPCODE
# Board class
class Board(): # My general assumption is the states and constraints of the game come here.
  # def __init__(self, states = np.zeros([2,3,3])):  # NEP When you DIY a board state, as you will with rootstate, you will need to identify moves made by "1"s. By default it's supposed to be empty board.
  # NEP This header is wrong because  np.zeros([2,3,3]) is only evaluated once when the class is defined, not when every instance is defined.
  def __init__(self, states=None):
    self.states = np.zeros([2,3,3]) if states == None else states
    self.player = 0 # it starts with one player and this variable can be toggled to the other.

  # copy of *current* board states
  def copying(self):
    copy = Board() # surprising to me that you can define an instance in a method of a class. But here we go.
    copy.states = np.copy(self.states)
    copy.player = self.player  # suprising to me that they do not do a copy here. # NEP I just worried that they would be modified together.
    # Verify copy
    # print("Original unoccupied:", self.get_unoccupied())
    # print("Copy unoccupied:", copy.get_unoccupied())
    return copy

  # board_state updating
  def updating(self, position):  # position: (x,y) represents position to be updated
    row, col = position
    if np.any(self.states[:, row, col]):  # if cell is occupied
      return  # position already taken
    self.states[self.player][row][col] = 1
    self.player ^= 1

  # get remaining moves (unoccupied) on the board
  def get_unoccupied(self):
    if self.resulting():  # We will not enter the function if result is declared.
      return []
    return np.argwhere(self.states[0] + self.states[1]==0).tolist() # both 0 --> np.any returns True --> np.argwhere returns indices of empty positions

  def resulting (self): # function for returning the results (win/lose/draw)
    # NEP I suspect that this is not a function, because result should be automatically decided, not called. But it might be if it's called every time you make a move.
    # NEP updating: this should be written, because each time you run simulation you'd need to return a result (True/False), that can in return updates the nodes in BP.

    previous_player_board = self.states[self.player^1]  # You have to toggle back to last player who has moved to check their result. You do this because you toggle player each time you update the board.

    rows = np.any(np.sum(previous_player_board,axis=1)==3)  # NEP I am not too sure whether "any" would work fine here bc it's three rows to be "any-ed"
    columns = np.any(np.sum(previous_player_board,axis=0)==3)
    diagonals = np.any(np.trace(previous_player_board)==3)
    reverse_diagonals = np.any(np.trace(np.flip(previous_player_board, axis=1))==3)
    self.reward =  rows or columns or diagonals or reverse_diagonals  # if any suffice, we would return a True, which will send the signal back to the nodes in BP
    return self.reward


In [149]:
# NEPCODE
# Node class --> A kind of treenode tailored for TTT game
class Node():
  def __init__(self, parent = None, board= None, idx = None):
    self.parent = parent
    self.children = []   # NEP i initially defined this in header but it seems you cannot define it as a *mutable* list in the header.
    self.visits = 0
    self.wins = 0
    self.board = board
    self.idx = idx
    self.untried = board.get_unoccupied() if board is not None else [] # a list of indices pointing to untravelled nodes.
    # NEP I think in extension/simulation/a copy of current board state, untravelled=unvisited. Bc in that simulation, you'd actually make moves on board state as you would with actual board.
    # NEP here we added if board is not None else [] bc it runs into a scenario of board=None. But I do not think this should happen per how new node is initiated (always a board assigned)
  # make a selection  Conditions: highest UCB1 score  # NEP my problem with this is it seems UCB1 should be better in MCTS.
  def select(self):
    for child in self.children:
        score = child.wins / child.visits + 0.2 * np.sqrt(2 * np.log(self.visits) / child.visits)
        print(f"Child {child.idx}: UCB1 score = {score}")

    s = sorted(self.children, key=lambda c:
               c.wins/c.visits + 0.2*np.sqrt(2*np.log(self.visits)/c.visits)
               if c.visits > 0 and self.visits > 0
               else float('inf')  # Unvisited nodes get infinite UCB1 value
               ) # NEP/GPT: Use of UCB1 requires defining a constant C for exploration, typically set to sqrt(2) in many MCTS implementations, rather than hardcoding 0.2.
    # if not s:
    #     return None  # or return random valid move
    return s[-1]  # the last of an ascending list, the next move of largest return.


  # connect a child to parent node. Input: current board, idx of untravelled node; Output: a connected child
  def expand(self, board, idx):
    child = Node(parent=self, board=board, idx=idx) # NEP to modify parent and board. They should be special to current child.
    self.untried.remove(idx)  # remove that from untravelled list.
    self.children.append(child) # Successfully extended the node into the list of children (to be selected)
    return child

  # for each node, there is win/visit num associated. Input: score from current run of MCTS
  def update(self, score):
    self.visits += 1
    self.wins += score


In [150]:
# NEPCODE

# MCTS # Specialized in MCTS strategy, not the game rules.

def MCTS(board=None, max_iter=None, idx=(-1, -1)):
  root = Node(board=board)  # current actor(node)
  iter = 0

  # each round is based on current board state/node
  while iter <= max_iter and len(board.get_unoccupied()) > 0:  # Conditions: smaller than max_iteration and still positions on board not tried.
                              # It was board.get_unoccupied() before, but an empty list is not None.
    iter += 1
    board = board.copying() # creating a new board copy when you start an iteration
    node = root # bc you will traverse to this copy of board state
    player = board.player  # When updating board state, the players will take turn.

    # Step 1 Selection: make a move / select a child on a copy of current states.
    while board.get_unoccupied() == [] and node.children != []: # Conditions: as long as the node is fully expanded and we have not reached terminal, we SELECT a child. Actually the more the better. You can pin down to the ternimal node by this approach. I mean why not it's based on UCB1 and it has all nodes as children.
      node = node.select()  # You traserse to best child on this copy state.
      board.updating(node.idx) #  NEP it troubles me here bc I did not see where possibly they have given an idx value to this node therefore you can just call it. But i guess it may just return None. In that case however, what's the point?
                                            # To answer my own question here. Yes it's defined. It's from self.children when SELECT. And any child idx is given and is randomly chosen fro untried moves.
    # Step 2 Expansion: the action of connecting to a untravelled node. Conditions: as long as there is untravelled position left
    if node.untried != []:  # if instead of why bc by setup you expand only one node a time.
      idx = random.choice(node.untried)
      board.updating(idx)
      node = node.expand(board=board.copying(), idx=idx) # Note that you create a new copy of board state for the child node that is seperate from current board state.

    # Step 3 Simulation: Going down from the newly expanded node to the terminal
    while board.get_unoccupied() !=[] and not board.resulting():
      board.updating(random.choice(board.get_unoccupied()))

    # Step 4 Backpropagation: return the stats to the parent node (how many steps and how many wins)
    while node:   # as long as there is a (parent) node we can come back to.
      won = board.resulting()
      if won:
        if board.player == player:  # NEP to figure out
          score = 1  # current player has won
        else:
          score =1 # current player has lost
      else:
        score = 0  # draw
      node.update(score)
      node = node.parent

  sorted_nodes = sorted(root.children, key=lambda c: c.wins/c.visits)
  # Safety check
  if not sorted_nodes:
      return None  # or return random valid move
  return  sorted_nodes[-1].idx # the move with highest score.


### Unit tests - MCTS TTT

In [153]:
import unittest
import numpy as np

class TestBoard(unittest.TestCase):
    def setUp(self):
        self.board = Board()

        # Verify board is initialized correctly
        self.assertTrue(np.all(self.board.states == 0), "Board should start empty")

        # Verify we can get unoccupied positions
        unoccupied = self.board.get_unoccupied()
        self.assertIsNotNone(unoccupied, "Should get unoccupied positions")
        self.assertEqual(len(unoccupied), 9, "Empty board should have 9 positions")

    def test_initialization(self):
        self.assertTrue(np.array_equal(self.board.states, np.zeros((2, 3, 3))))
        self.assertEqual(self.board.player, 0)

    def test_copying(self):
        board_copy = self.board.copying()
        self.assertNotEqual(id(self.board.states), id(board_copy.states))
        self.assertEqual(self.board.player, board_copy.player)

    def test_updating(self):
        self.board.updating((1, 1))
        self.assertEqual(self.board.states[self.board.player ^ 1][1][1], 1)
        self.board.updating((1, 1))  # should do nothing since occupied
        self.assertEqual(self.board.states[self.board.player ^ 1][1][1], 1)

    def test_get_unoccupied(self):
        self.board.updating((1, 1))
        unoccupied = self.board.get_unoccupied()
        self.assertNotIn([1, 1], unoccupied)
        self.assertEqual(len(unoccupied), 8)

    def test_expansion(self):
        # Initial node setup with board
        self.node = Node(board=self.board)

        # Ensure untried moves exist
        self.assertTrue(len(self.node.untried) > 0,
                        f"No untried moves. Board state: {self.board.states}")

        idx = self.node.untried[0]
        new_child = self.node.expand(board=self.board.copying(), idx=idx)
        self.assertIn(new_child, self.node.children)
        self.assertNotIn(idx, self.node.untried)


class TestNode(unittest.TestCase):
    def setUp(self):
        self.board = Board()
        self.node = Node(board=self.board)

    def test_initialization(self):
        self.assertEqual(self.node.visits, 0)
        self.assertEqual(self.node.wins, 0)
        self.assertEqual(self.node.children, [])
        self.assertEqual(self.node.untried, self.board.get_unoccupied())

    def test_expansion(self):
        idx = self.node.untried[0]
        new_child = self.node.expand(board=self.board.copying(), idx=idx)
        self.assertIn(new_child, self.node.children)
        self.assertNotIn(idx, self.node.untried)

    def test_selection(self):
        # Initialize child nodes with mock idx and UCB1-related values
        child1 = Node(parent=self.node, idx=(0, 0), board=self.board)
        child2 = Node(parent=self.node, idx=(1, 1), board=self.board)

        child1.wins, child1.visits = 10, 20
        child2.wins, child2.visits = 5, 10

        self.node.children = [child1, child2]
        self.node.visits = 30  # total visits should be sum of children
        selected = self.node.select()

        self.assertEqual(selected, child2, "Expected the child with highest UCB1 score to be selected.")
    '''
    def test_selection(self):
      # Assuming self.node has children added and `child1` should have the highest UCB1 score
      selected = self.node.select()

      # Manually calculate expected highest UCB1 score for validation
      expected_scores = [
          child.win / child.visit + 0.2 * np.sqrt(2 * np.log(self.node.visit) / child.visit)
          for child in self.node.children
      ]
      highest_score_index = expected_scores.index(max(expected_scores))
      highest_score_child = self.node.children[highest_score_index]

      # Add a print statement to confirm selected node
      print(f"Expected highest child idx: {highest_score_child.idx}, Selected child idx: {selected.idx}")

      # Assert selected node matches expected highest score node
      self.assertEqual(selected, highest_score_child)
    '''

    def test_backpropagation(self):
        self.node.update(1)
        self.assertEqual(self.node.visits, 1)
        self.assertEqual(self.node.wins, 1)


class TestMCTS(unittest.TestCase):
    def setUp(self):
        self.board = Board()

    def test_mcts_execution(self):
        result = MCTS(board=self.board.copying(), max_iter=10)
        self.assertIsNotNone(result)

    def test_best_move_selection(self):
        best_move = MCTS(board=self.board.copying(), max_iter=100)
        self.assertIn(best_move, [idx for idx in self.board.get_unoccupied()])

    def test_simulated_game(self):
        board = Board()
        while not board.resulting():
            best_move = MCTS(board=board.copying(), max_iter=100)
            board.updating(best_move)
        self.assertTrue(board.resulting(), "Game should end with a result.")

if __name__ == "__main__":
    unittest.main(argv=[''], exit=False)


............
----------------------------------------------------------------------
Ran 12 tests in 0.067s

OK


Child (0, 0): UCB1 score = 0.616639571015366
Child (1, 1): UCB1 score = 0.6649532632393105


## Annotation
https://stackoverflow.com/questions/49456415/monte-carlo-tree-search-tic-tac-toe-poor-agent

In [ ]:
class Board: # So in class Board, you want to define a function to return a copy (for simulation purpose), make the move that reflects  the change over the board, maintain a storage for untraveled nodes, and get result
  '''
  class handling state of the board
  '''

  def __init__(self):
    self.states = np.zeros([2,3,3])  # 忘记了初始要用np.zeros
    self.player = 0 # current player's turn  # remember thay they use  0 & 1 as a switcher for players to take turn. Cleaver. You can unify everything else accordingly. board states and positions of nodes can be just in one array.

  def copy(self):   # You create a copy of current state/player on which you may run a simulation. It's like an instance of board, but with player/state info maintained.
    '''
    make copy of the board
    '''
    copy = Board()
    copy.player = self.player
    copy.states = np.copy(self.states)
    return copy

  '''
  def move(self, move):  # move para better be "position", of [x,y] shape

    take move of form [x,y] and play
    the move for the current player
    if np.any(self.state[:,move[0],move[1]]): return  # if occupied, then you don't occupy it again.
    self.state[self.player][move[0],move[1]] = 1  # NEP smart. I was confused how can they find a position to modify. This is how.
    self.player ^= 1   # togging betw two states; XOR operation. Clever.
  '''

# modifying board state: from an idx to a changed board state.
def make_move(self, position):  # position: of [x,y] shape
    '''
    Places a marker at the specified board position [row, col]
    for the current player if the position is empty
    '''
    row, col = position
    if np.any(self.states[:, row, col]):  # if cell is occupied
        return  # position already taken
    self.states[self.player][row, col] = 1  # NEP smart. I was confused how can they find a position to modify. This is how.
    self.player ^= 1  # togging betw two states; XOR operation. Clever.


# get remaining untravelled locations.
def get_moves(self):  # Seems to miss the scenarios where game is won and over.
  '''
  return remaining possible board moves
  (ie where there are no O's or X's)
  '''
  if self.check_for_win():
    return []  # ensure that nobody has won yet
  return np.argwhere(self.states[0]+self.states[1]==0).tolist()  # np.argwhere: Find the indices of array elements that are non-zero (True values in a boolean array), grouped by element.
  # NEP it will return a list of indices (x,y) indicating positions both empty.

  '''

  def result(self):

    check rows, columns, and diagonals
    for sequence of 3 X's or 3 O's
    board = self.state[self.player^1]  # This guy just uses a load of confusing naming. Cannot you fucking just have a new name?
    col_sum = np.any(np.sum(board,axis=0)==3)
    row_sum = np.any(np.sum(board,axis=1)==3)
    d1_sum  = np.any(np.trace(board)==3)  # Return the sum along diagonals of the array.
    d2_sum  = np.any(np.trace(np.flip(board,1))==3) # Flip an array horizontally (axis=1).
    return col_sum or row_sum or d1_sum or d2_sum  # Return a True if any of those is true otherwise False.
    '''

def check_for_win(self):
    previous_player_board = self.states[self.player^1]  # or last_move_board

    has_winning_column = np.any(np.sum(previous_player_board, axis=0) == 3)
    has_winning_row = np.any(np.sum(previous_player_board, axis=1) == 3)
    has_winning_diagonal = np.any(np.trace(previous_player_board) == 3)
    has_winning_antidiagonal = np.any(np.trace(np.flip(previous_player_board, 1)) == 3)

    return has_winning_column or has_winning_row or has_winning_diagonal or has_winning_antidiagonal

In [ ]:
# Annotating other's code

class Node: # For a class Node, you'd want its parent (for returning stats), its children (for deciding best node), maintaining/updating wins & visits, act to select best child, expand to an untraveled node,
  '''
  maintains state of nodes in
  the monte carlo search tree
  '''
  def __init__(self, parent=None, idx_move=None, board=None):  # I do not know what is an action
    self.parent = parent  # NEP why. It's just current node. It turns out that TreeNode just defines a node as None, but with attributes and operations within __init__, and ofc methods.
    self.board = board
    self.children = []  # adding a new child node (default, but also based on currect action & board)
    self.wins = 0
    self.visits = 0
    self.untried_moves = board.get_moves().tolist()  # list of indices pointing to untraveled nodes
    # Note though, node.untried moves != board.get_moves().
    # The former is updated when you expand to a new node in simulation
    # whereas the latter remains faithful to untravelled positions on current board state.

    self.idx_move = idx_move # I hate his code! But this should be an idx pointing to a position (since it can be removed from list of untravaled_nodes)

  def select(self):
    '''
    select child of node with
    highest UCB1 value
    '''
    s = sorted(self.children, key=lambda c:c.wins/c.visits+0.2*np.sqrt(2*np.log(self.visits)/c.visits))
    return s[-1] # by default, last is the largest. So select will choose a move that has the most return among all children of a node.

  def expand(self, idx_move, board): # here you pass on a seperate board state , idx_move because you want to pass on them to define the child node.
    '''
    expand parent node (self) by adding child
    node with given action and state
    '''
    child = Node(parent=self, idx_move=idx_move, board=board)  # NEP Interesting. This was one of my mental blocks. It turns out that you can reference a Node from within.
    self.untried_moves.remove(idx_move)
    self.children.append(child)
    return child

  def update(self, result):
    self.visits += 1
    self.wins += result

In [ ]:
# SO MCTS algo is a function, just like a remote cheating super computer, which, based on current board state, once you click, return you a move rec.
def UCT(rootstate, maxiters):  # to give a rec at a given place, you run mixiters of simulation atmost.

  root = Node(board=rootstate) # I think the rootstate is an instance that refers to a board state where you've made all actual moves until now.

  for i in range(maxiters):   # NEP case is now clear. You just need to create a whole board state each round of simulation.
    node = root
    board = rootstate.copy() # and for each board state on which you wanted to make simulations,  you make # of simulations & board states.

    # selection (exploitation) - select best child if parent fully expanded and not terminal
    # NEP i have not gotten the while condition here.
    while node.untried_moves == [] and node.children != []:
      node = node.select()
      board.make_move(node.idx_move)  # NEP I cannot see where they got this node.idx_move from. Possible it's just from children, which is from rootstateboard.


    # another mental bottleneck is whether it's ok to expand the board for just once as it is done here.
    # expansion (exploration) - expand parent to a random untried action, modify that board position, and connect parent to this new child
    if node.untried_moves != []:  # as long as there are untried moves in this simulation you can expand it once
      a = np.random.choice(node.untried_actions)  # one item is an idx of a random move  The code is terribly. Cannot you even properly name your variables.
      board.move(a)
      node = node.expand(a, board.copy())
      # NEP when you EXPAND, you make a random untraveled move, you mark that move in this iteration of board copy, and add that node in current node's children list.

    # simulation - rollout to terminal state from current
    # state using random actions
    while board.get_moves() != [] and not board.result():  # as long as there are still unoccupied positions you can move this copy
      board.move(np.random.choice(board.get_moves()))
      # NEP: in simulation you roll each board state + path to its terminal (meaning you get the final result)

    # backpropagation - propagate result of rollout game up the tree
    # reverse the result if player at the node lost the rollout game
    '''
    while node != None:
      result = board.result()
      if result:
        if node.board.player==board.player:
          result = 1  # Fuck you. Use a different name.
        else: result = -1 # current player loses the game here.
      else: result = 0
      node.update(result)
      '''

    while node is not None:
      has_won = board.result()  # or is_game_won, has_winning_line
      if has_won:
          if node.board.player == board.player:
              score = 1    # current player wins
          else:
              score = -1   # current player loses
      else:
          score = 0       # game continues/draw
      node.update(score)
      node = node.parent # each time you return to the parent node until None.
      # NEP for a current board (some traveled some untraveled), you update each node whether it would win the game or not (result=1,0,-1) if continued
  s = sorted(root.children, key=lambda c:c.wins/c.visits)
  return s[-1].action   # So it seems each time you run a UCT, which is a MCTS-based strategy and return rec of next move, instead of playing the whole game as I thought.